In [ ]:
import pandas as pd
import json
from pprint import pprint

def read_file(file_name):
    '''
    This function will read the file containing tweet ids sperating by lines (in groups)
    '''
    all_ids = pd.read_table(file_name,sep='\n',header=None)
    return all_ids

In [ ]:
train_ids = read_file('../../project-data/train.data.txt')
# train_ids

In [ ]:
dev_ids = read_file('../../project-data/dev.data.txt')
# dev_ids

In [ ]:
covid_ids = read_file('../../project-data/covid.data.txt')
# covid_ids

### Get valid data ids

In [ ]:
def get_new_data_ids(ids_file, data_all_file, output):
    original_ids = read_file(ids_file)
    
    tweets = pd.read_json(data_all_file)
    
    valid_ids = original_ids.iloc[tweets.keys()]
    
    valid_ids.to_csv(output, index=False, header=False)

In [ ]:
get_new_data_ids('../../project-data/train.data.txt', "data/train_data_all.json", "data/valid.train.ids.txt")

In [ ]:
get_new_data_ids('../../project-data/dev.data.txt', "data/dev_data_all.json", "data/valid.dev.ids.txt")

In [ ]:
get_new_data_ids('../../project-data/covid.data.txt', "data/covid_data_all.json", "data/valid.covid.ids.txt")

### Split the data into separate tweet json file

In [ ]:
def split_file(file, output_folder_dir):
    tweets = pd.read_json(file)
    tweets = tweets.T.reset_index(drop=True)
    
    # get all users info in a dict by author id
    users_info = tweets['includes'].map(lambda x: {n['id']: n for n in x['users']})
    users_dict = {}
    for i in range(len(users_info)):
        users_dict.update(users_info[i])
        
    # get all tweets info
    tweet_list = []
    for i in range(len(tweets)):
        tweet_list.extend(tweet for tweet in tweets['data'][i])
        
    # add user info to tweet info by author id
    # and then output each tweet into json file
    for tweet in tweet_list:
        tweet['user'] = users_dict[tweet['author_id']]
        
        with open("{}/{}.json".format(output_folder_dir, tweet['id']), "w",encoding='utf-8') as outfile:
            json.dump(tweet, outfile)

In [ ]:
split_file("data/dev_data_all.json", "data/dev_data")

In [ ]:
split_file("data/train_data_all.json", "data/train_data")

In [ ]:
split_file("data/covid_data_all.json", "data/covid_data")